# Boosted trees verses random forest


Compare LightGBM, catbost, sklearn's GradientBoostingRegressor, with sklearn's Random Forest<br>
LightGBM is a microsoft gradient boosted tree product<br>
catboost is a Yandex gradient boosted tree product<br>

See <a href="https://towardsdatascience.com/catboost-vs-light-gbm-vs-xgboost-5f93620723db">CatBoost vs. Light GBM vs. XGBoost</a> for relative comparisons<br>
See <a href="https://towardsdatascience.com/boosting-showdown-scikit-learn-vs-xgboost-vs-lightgbm-vs-catboost-in-sentiment-classification-f7c7f46fd956">Scikit-Learn vs XGBoost vs LightGBM vs CatBoost in Sentiment Classification</a> for another relative comparison.


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

## Install LightGBM
conda and pip both have the same version (3.3.2) and the last upload was 3 months ago. This package is currently being maintained. Prefer conda so anaconda can coordinate LightGBMs dependencies with all other conda packages

In [7]:
# !conda install -c conda-forge lightgbm -y
import lightgbm
lightgbm.__version__

'3.1.1'

## Install catboost
conda and pip both have the same version (1.0.4) and the last upload was 2 months ago. This package is currently being maintained. Prefer conda so anaconda can coordinate catboosts dependencies with all other conda packages

In [6]:
# !conda install -c conda-forge catboost -y
import catboost
catboost.__version__

'1.0.4'

## Regression

### Data

In [10]:
from sklearn.datasets import fetch_california_housing
calif_housing = fetch_california_housing()

for line in calif_housing.DESCR.split("\n")[5:22]:
    print(line)

calif_housing_df = pd.DataFrame(data=calif_housing.data, columns=calif_housing.feature_names)
calif_housing_df["Price($)"] = calif_housing.target

calif_housing_df.head()

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price($)
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [11]:
#get train/test split
from sklearn.model_selection import train_test_split
X, y = calif_housing.data, calif_housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8,test_size=0.2,random_state=123)

### R squared - a way to qualify a models predictions

The following regressors use R squared as the default objective to optimize.  See <a href="https://www.youtube.com/watch?v=2AQKmw14mHM">Statquest: R-squared, Clearly Explained!!!</a> for a great explanation plus examples.

Usually 0<R squared<1  .  It ranges between these 2 values and is interpreted as how well the model fits the data. (In statistics this is called explained variance)

If R squared =0,the line fitted to data is no more accurate than taking the mean of the data.<br>
If R squared =1,the line fitted to the data is a perfect match<br>
If R squared is negative then the line fitted to the data is a worse fit than just taking the average value of the data.

In [103]:
#It was not clear what objective lightGBM optimizes
#so I implemented R squared below 
def rsquared(preds, y):
    RSS=np.sum(np.square(preds-y))
    ymean=np.sum(y)/len(y)
    TSS=np.sum(np.square(y-ymean))
    return 1-RSS/TSS

def scoremodel(clf,X_test, y_test):
    print("Score on test set: {:.2f}".format(clf.score(X_test, y_test)))
    #run score using rsquared function above
    preds=clf.predict(X_test)
    rsq=rsquared(preds,y_test)
    print("Score on test set using rsquared: {:.2f}".format(rsq))
    

### random forest- default hyperparameters

In [105]:
%%time
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(random_state=42)
_=clf.fit(X_train, y_train)

CPU times: user 6.85 s, sys: 35.7 ms, total: 6.88 s
Wall time: 6.88 s


In [106]:
scoremodel(clf,X_test, y_test)

Score on test set: 0.81
Score on test set using rsquared: 0.81


### lightgbm- default hyperparameters

In [107]:
%%time
from lightgbm import LGBMRegressor
clf = LGBMRegressor(random_state=42)
_=clf.fit(X_train, y_train)

CPU times: user 3.17 s, sys: 2.91 ms, total: 3.17 s
Wall time: 227 ms


In [108]:
scoremodel(clf,X_test, y_test)

Score on test set: 0.84
Score on test set using rsquared: 0.84


### catboost -default parameters

In [109]:
%%time
from catboost import CatBoostRegressor
clf = CatBoostRegressor(silent=True, random_state=42)
_=clf.fit(X_train, y_train)

CPU times: user 15.2 s, sys: 1.38 s, total: 16.6 s
Wall time: 1.4 s


In [110]:
scoremodel(clf,X_test, y_test)

Score on test set: 0.86
Score on test set using rsquared: 0.86


## Classification

### Data

In [194]:
#took 10% of original dataset, dropped bunch of columns
data = pd.read_csv("../datasets/kaggle/flights/flights_small.csv")
# data.head()

In [195]:
#only later than 10 minutes is considered late
data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1

#convert to ordinal (even though they are categorical)
cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1

#unlbalanced dataset, make sure you get a stratified sample
X_train, X_test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"], stratify=data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)

In [190]:
# data.head()
# X_test.shape
y_test.value_counts()

0    111559
1     31276
Name: ARRIVAL_DELAY, dtype: int64

In [166]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

def score_classifier_model(clf,X_test, y_test):
    res = clf.predict(X_test)
    print (classification_report(y_test, res))
    print(confusion_matrix(y_test,res))
    

### random forest- default hyperparameters

In [167]:
%%time
from sklearn.ensemble import RandomForestClassifier
clfc = RandomForestClassifier(random_state=42)
_=clfc.fit(X_train, y_train)

CPU times: user 1min 4s, sys: 155 ms, total: 1min 4s
Wall time: 1min 4s


In [168]:
score_classifier_model(clfc,X_test, y_test)

              precision    recall  f1-score   support

           0       0.81      0.98      0.89    111559
           1       0.71      0.17      0.28     31276

    accuracy                           0.80    142835
   macro avg       0.76      0.58      0.58    142835
weighted avg       0.79      0.80      0.75    142835

[[109344   2215]
 [ 25809   5467]]


In [ ]:
# macro avg (precision)   =(.81+.71)/2
# weighted avg (precision)= (111559/142835)*.81 + ((31276/142835)*.71)

### lightgbm- default hyperparameters

In [169]:
%%time
from lightgbm import LGBMClassifier
clf_lgbm = LGBMClassifier(random_state=42)
_=clf_lgbm.fit(X_train, y_train)

CPU times: user 23.9 s, sys: 307 ms, total: 24.2 s
Wall time: 1.7 s


In [170]:
score_classifier_model(clf_lgbm,X_test, y_test)

              precision    recall  f1-score   support

           0       0.80      0.99      0.88    111559
           1       0.74      0.12      0.20     31276

    accuracy                           0.80    142835
   macro avg       0.77      0.55      0.54    142835
weighted avg       0.79      0.80      0.73    142835

[[110268   1291]
 [ 27610   3666]]


### catboost -default parameters

In [171]:
%%time
from catboost import CatBoostClassifier
clf_catboost = CatBoostClassifier(silent=True, random_state=42)
_=clf_catboost.fit(X_train, y_train)

CPU times: user 4min 20s, sys: 7.33 s, total: 4min 28s
Wall time: 21.2 s


In [172]:
score_classifier_model(clf_catboost,X_test, y_test)

              precision    recall  f1-score   support

           0       0.81      0.98      0.89    111559
           1       0.72      0.19      0.31     31276

    accuracy                           0.81    142835
   macro avg       0.77      0.59      0.60    142835
weighted avg       0.79      0.81      0.76    142835

[[109215   2344]
 [ 25226   6050]]


## Notice that catboost consistantly outperforms Random Forest and LightGBM for these 2 tasks?  See the articles mentioned in first cell for further evidence.